<img align='left' src = https://www.linea.org.br/brand/linea-logo-color.svg width=150 style='padding: 20px'> 

## Photo-z Training Set

Collection of public redshift catalogs made available by spectroscopic surveys combined with photometric data from DES DR2.


Contact: Julia Gschwend ([julia@linea.org.br](mailto:julia@linea.org.br)) <br>
Last verified run: **2024-Jul-22** <br>


#### Acknowledgments
If you use this dataset to generate scientific results, please add a reference to [Gschwend et al., 2018](https://ui.adsabs.harvard.edu/abs/2018A%26C....25...58G/abstract) and acknowledge LIneA in the acknowledgments section of your publication. For instance:

'_This research used computational resources from the Associação Laboratório Interinstitucional de e-Astronomia (LIneA) with the financial support of INCT do e-Universo (Process no. 465376/2014-2)._'

#### Notes 

The training set was created based on the spatial correspondence between the objects present in the redshift catalog described above and the object table (_coadd_objects_) of DES DR2, with a search radius of 1.0 _arcsec_. The aim was to include the columns of the set photometric measurements that are useful for calculating photo-z (apparent magnitudes and their respective errors).

The characterization of the spectroscopic redshifts catalog is available in a [separate notebook](https://github.com/linea-it/pzserver/blob/main/docs/notebooks/public-specz-compilation.ipynb). If you have questions, please get in touch with us.

--- 


## DES DR2

Please find details about DES and its second data release on:  
    
- [DES website](https://www.darkenergysurvey.org/)
- [DES Data Management website](https://des.ncsa.illinois.edu/home)
- [DES DR2 paper](https://arxiv.org/abs/2101.05765)

---

Imports and configs

In [ ]:
# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
#from astropy.units.quantity import Quantity

# Bokeh
import bokeh
from bokeh.io import output_notebook, show, output_file, reset_output
#from bokeh.models import ColumnDataSource, Range1d, HoverTool
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, ColorBar
from bokeh.models import CDSView, GroupFilter
from bokeh.plotting import figure, show, gridplot, output_notebook
from bokeh.models import Range1d, LinearColorMapper, ColorBar
from bokeh.transform import factor_cmap
from bokeh.plotting import show
output_notebook()

# HoloViews
import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import datashade, dynspread
from holoviews.plotting.util import process_cmap

# PZ Server
from pzserver import PzServer
with open('token.txt', 'r') as file:
    token = file.read()
pz_server = PzServer(token=token, host="pz-dev") # "pz-dev" is the temporary host for test phase  

# Configs
import warnings
warnings.filterwarnings('ignore')
sns.set(color_codes=True, font_scale=1.5) 
sns.set_style('whitegrid')
plt.rcParams.update({'figure.max_open_warning': 0})
hv.extension('bokeh')
%reload_ext autoreload 
%autoreload 2 
%matplotlib inline 

In [ ]:
print('Python version: ' + sys.version)
print('Numpy version: ' + np.__version__)
print('Bokeh version: ' + bokeh.__version__)
print('HoloViews version: ' + hv.__version__)

## Retrieve data 

Auxiliary file: `des-round19-poly.txt` (contours of the area covered by the survey, i.e., DES _footprint_, 2019 version) 

Download the file from the repository [kadrlica/skymap](https://github.com/kadrlica/skymap/blob/master/skymap/data/des-round19-poly.txt) on GitHub:

In [ ]:
! wget https://raw.githubusercontent.com/kadrlica/skymap/master/skymap/data/des-round19-poly.txt  

Read DES footprint file `des-round19-poly.txt`:

In [ ]:
foot_ra, foot_dec = np.loadtxt('des-round19-poly.txt', unpack=True)
foot_coords = SkyCoord(ra=-foot_ra*u.degree, dec=foot_dec*u.degree, frame='icrs')
foot_df = pd.DataFrame({'foot_ra': np.array(foot_coords.ra.wrap_at(180*u.degree)), 
                        'foot_dec': np.array(foot_coords.dec)})

Retrieve training set from PZ Server 

In [ ]:
training_set_obj = pz_server.get_product('27_public_training_set_des_dr2')

In [ ]:
training_set_obj.display_metadata()

In [ ]:
training_set = training_set_obj.data

In [ ]:
type(training_set)

In [ ]:
assert len(training_set) == 592493

In [ ]:
training_set.info(memory_usage="deep")

In [ ]:
training_set.head()

Meaning of columns:

| Column name | Meaning |
|--:|:--|
| **coadd_object_id**| Unique object identifier in the DES DR2 photometric catalog (_coadd_objects_ table). |
| **ra** | Right Ascension (degrees) |
| **dec** | Declination (degrees) |
| **z** | Redshift |
| **err_z** | Redshift error. When unavailable, replaced by 99.0 |
| **flag_des**| Standardized quality marker (details [above](#flags))|
| **survey** | Name of the project or survey of origin. |
| **flag_survey** | Original quality flag given by the origin survey. |
| **mag\_auto\_[g,r,i,z,y]\_dered** | Apparent magnitude in bands [g, r, i, z, y], corrected for reddening |
| **magerr\_auto\_[g,r,i,z,y]** | Apparent magnitude error in bands [g, r, i, z, y] |

Compute colors $(g-r)$ e $(r-i)$ 

In [ ]:
training_set['gmr'] = training_set['mag_auto_g_dered'] - training_set['mag_auto_r_dered']
training_set['rmi'] = training_set['mag_auto_r_dered'] - training_set['mag_auto_i_dered']

Basic statistics 

In [ ]:
training_set.describe()


--- 

## Sample characterization

Check below a brief characterization of the data contained in the compiled collection of spectroscopic catalogs.

In [ ]:
frac = 0.06
train_sample_for_plots = training_set.sample(frac=frac, axis='index')
assert len(train_sample_for_plots) == round(frac * len(training_set))
print(len(train_sample_for_plots))
train_sample_for_plots = training_set # comment this line to use a fraction of the sample 

--- 

#### Spatial Distribution 


In [ ]:
coords = SkyCoord(ra=-np.array(train_sample_for_plots.ra)*u.degree, 
                  dec=np.array(train_sample_for_plots.dec)*u.degree, frame='icrs')
train_sample_for_plots.ra = np.array(coords.ra.wrap_at(180*u.degree))
train_sample_for_plots.dec = np.array(coords.dec)

In [ ]:
%%time
fig = plt.figure(figsize=[14,6])
ax = fig.add_subplot(111, projection='mollweide')   
ra_rad = coords.ra.wrap_at(180 * u.deg).radian
dec_rad = coords.dec.radian
plt.plot(ra_rad, dec_rad, '.', alpha=0.1)
plt.plot(-np.radians(foot_ra), np.radians(foot_dec), '-', color='darkorange')
org=0.0
tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
tick_labels = np.remainder(tick_labels+360+org,360)
ax.set_xticklabels(tick_labels)     # we add the scale on the x axis
ax.set_xlabel('R.A.')
ax.xaxis.label.set_fontsize(14)
ax.set_ylabel('Dec.')
ax.yaxis.label.set_fontsize(14)
ax.grid(True)
plt.tight_layout()

Redshift distribution

In [ ]:
redshift = hv.Dimension('z', label='spec-z', range=(0.0, 2.0))
(count, z_bin) = np.histogram(train_sample_for_plots.z, bins='fd')

In [ ]:
z_distribution = hv.Histogram((count, z_bin), kdims=redshift).opts(
    title='Distribuição de redshifts', xlabel='spec-z', height=400, width=800)   
z_distribution

#### Quality Flags

In [ ]:
training_set.flag_des.value_counts() 

In [ ]:
def fmt(x):
    return '{:.1f}%'.format(x)
counts = pd.DataFrame(data={'flag_des':[len(training_set.query('flag_des ==3')), 
                                        len(training_set.query('flag_des ==4'))]}, index= [3, 4])
counts.plot.pie(y='flag_des', labels=None, autopct=fmt, colors=['darkorange', 'steelblue']) 
counts

Redshift distributions depending on the quality flag

In [ ]:
(count4, z_bin4) = np.histogram(train_sample_for_plots.query('flag_des == 4').z, bins='fd')
z_distribution4 = hv.Histogram((count4, z_bin4), kdims=redshift).opts(
    title='flag_des = 4', xlabel='spec-z', height=400, width=400, xlim=(0., 2.))
(count3, z_bin3) = np.histogram(train_sample_for_plots.query('flag_des == 3').z, bins='fd')
z_distribution3 = hv.Histogram((count3, z_bin3), kdims=redshift).opts(
    title='flag_des = 3',  color='darkorange', xlabel='spec-z', height=400, width=400, xlim=(0., 2.))
z_dist_by_flag = z_distribution4.options(height=350, width=450)  +  z_distribution3.options(height=350, width=450)             
z_dist_by_flag

#### Characteristics of the photometric sample

##### Magnitude distributions and their respective errors

In [ ]:
bands = ['g', 'r', 'i', 'z', 'y']

In [ ]:
fig = plt.figure(figsize=[12,4])
plt.subplot(1,2,1)
for band in bands:
    plt.hist(train_sample_for_plots.query(f'mag_auto_{band}_dered != 99.')[f'mag_auto_{band}_dered'], 
             bins=30, histtype='step', lw=2, log=True)
plt.xlabel('magnitude')
plt.ylabel('counts')
plt.xlim(12,28)
plt.ylim(10,)
plt.subplot(1,2,2)
for band in bands:
    plt.hist(train_sample_for_plots.query(f'mag_auto_{band}_dered != 99. & magerr_auto_{band} < 1.')[f'magerr_auto_{band}'], 
             bins=30, label=band, histtype='step', lw=2, log=True)
plt.xlabel('magnitude error')
plt.ylabel('counts')
plt.xlim(0,1)
plt.ylim(10,)
plt.legend(loc='upper right')
plt.tight_layout()

##### Magnitude errors

In [ ]:
plt.figure(figsize=[18,4])
for i, band in enumerate(bands): 
    plt.subplot(int(f'15{str(i+1)}'))
    query = f'mag_auto_{band}_dered != 99. & magerr_auto_{band} < 2.'
    plt.plot(train_sample_for_plots.query(query)[f'mag_auto_{band}_dered'],
             train_sample_for_plots.query(query)[f'magerr_auto_{band}'], 
             '.', alpha=0.3, color='steelblue')
    plt.xlabel(f'mag {band}')
    if i == 0: 
        plt.ylabel('error')
    plt.xlim(16, 28)    
    plt.ylim(0, 2)
    plt.tight_layout()

##### Magnitude X redshift

In [ ]:
clean = 'magerr_auto_i < 0.1 & mag_auto_g_dered != 99. & mag_auto_r_dered != 99. & mag_auto_i_dered != 99.'
train_sample_for_plots.query(clean, inplace=True)

In [ ]:
mag_vs_z = hv.Scatter(train_sample_for_plots[['z', 'mag_auto_i_dered']]).opts(
        toolbar='above', tools=['hover'], height=400, width=800, alpha=0.5, 
        size=2, xlim=(0,2), ylim=(14,24), xlabel='spec-z', ylabel='mag i')
mag_vs_z

##### CMD and color-color plots

In [ ]:
plot_style_bkh = dict(alpha=0.2,# color='steelblue',
                      marker='triangle', size=3,
                      xticks=5, yticks=5,
                      height=400, width=400,
                      toolbar='above')
plot_style = plot_style_bkh

In [ ]:
points = train_sample_for_plots

In [ ]:
imag = hv.Dimension('mag_auto_i_dered', label='mag i', range=(12, 24))
gmr = hv.Dimension('gmr', label='(g-r)', range=(-0.8, 3.0))
col_mag = hv.Scatter(points, kdims=imag, vdims=gmr).opts(**plot_style)
col_mag = col_mag.hist(dimension=[imag, gmr], num_bins=100, adjoin=True)

In [ ]:
rmi = hv.Dimension('rmi', label='(r-i)', range=(-0.8, 2.5))
gmr = hv.Dimension('gmr', label='(g-r)', range=(-0.8, 3.5))
col_col = hv.Scatter(points, kdims=rmi, vdims=gmr).opts(**plot_style)
col_col = col_col.hist(dimension=[rmi, gmr], num_bins=100, adjoin=True)

In [ ]:
col_mag + col_col